In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Rescaling, Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers, callbacks
import time, json, os, shutil, argparse
import warnings
warnings.simplefilter("ignore")

In [2]:
epochs = 100
batch_size = 160
labels = ["0", "1"]
dir_models = "./Models/HC/OG"
dir_training = "./Data/HC/OG/"
dir_validation = "./Data/HC/O/Validation"
img_width, img_height, img_depth = 320, 240, 3
path_model = os.path.join(dir_models, 'resnet.h5')
path_history = os.path.join(dir_models, 'resnet.csv')

In [3]:
resnet_50 = tf.keras.applications.resnet50.ResNet50(include_top=False, weights='imagenet')
resnet_50.trainable=False

inputs = tf.keras.Input(shape=(img_width, img_height, img_depth))
x = resnet_50(inputs)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dense(1)(x)
outputs = tf.keras.layers.Activation('sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs, name="ResNet50_TL")
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Nadam(learning_rate=1e-4, epsilon=1e-08),
              metrics=['accuracy'])
#model.summary()

In [4]:
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

train_generator = train_datagen.flow_from_directory(
    dir_training,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42)

validation_generator = test_datagen.flow_from_directory(
    dir_validation,
    target_size=(img_width, img_height),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42)

Found 320 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [5]:
callback_log = callbacks.CSVLogger(path_history, append=True)
callback_model_save = callbacks.ModelCheckpoint(filepath=path_model, save_weights_only=False, monitor='val_accuracy', mode='max', save_best_only=True)

In [6]:
time_in = time.time()

hist = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[callback_log, callback_model_save])

time_out = time.time()
print('\n', 'Time cost:', '\n', time_out-time_in)

Epoch 1/100
2/2 [==============================] - 53s 26s/step - loss: 4.7351 - accuracy: 0.5094 - val_loss: 5.7004 - val_accuracy: 0.5000
Epoch 2/100
2/2 [==============================] - 49s 26s/step - loss: 2.5499 - accuracy: 0.5625 - val_loss: 0.6218 - val_accuracy: 0.5500
Epoch 3/100
2/2 [==============================] - 52s 26s/step - loss: 0.4530 - accuracy: 0.7937 - val_loss: 0.5247 - val_accuracy: 0.8000
Epoch 4/100
2/2 [==============================] - 46s 23s/step - loss: 0.3736 - accuracy: 0.8562 - val_loss: 0.4953 - val_accuracy: 0.8000
Epoch 5/100
2/2 [==============================] - 41s 20s/step - loss: 0.3074 - accuracy: 0.8938 - val_loss: 0.6138 - val_accuracy: 0.7000
Epoch 6/100
2/2 [==============================] - 40s 21s/step - loss: 0.3742 - accuracy: 0.7875 - val_loss: 0.5308 - val_accuracy: 0.8000
Epoch 7/100
2/2 [==============================] - 40s 21s/step - loss: 0.2164 - accuracy: 0.9000 - val_loss: 0.4699 - val_accuracy: 0.7500
Epoch 8/100
2/2 [===

2/2 [==============================] - 49s 24s/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.5506 - val_accuracy: 0.7000
Epoch 60/100
2/2 [==============================] - 41s 20s/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.5496 - val_accuracy: 0.7000
Epoch 61/100
2/2 [==============================] - 39s 20s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.5480 - val_accuracy: 0.7000
Epoch 62/100
2/2 [==============================] - 43s 23s/step - loss: 9.4928e-04 - accuracy: 1.0000 - val_loss: 0.5490 - val_accuracy: 0.7000
Epoch 63/100
2/2 [==============================] - 49s 25s/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 0.5446 - val_accuracy: 0.7000
Epoch 64/100
2/2 [==============================] - 41s 20s/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.5431 - val_accuracy: 0.7000
Epoch 65/100
2/2 [==============================] - 39s 20s/step - loss: 8.9546e-04 - accuracy: 1.0000 - val_loss: 0.5412 - val_accuracy: 0.7000
Epoch 66/100
2/2 [